**Run every code block cell once, There are total 4 blocks.**

### Install Dependencies

This cell installs all required Python packages for interacting with
Google Docs and Google authentication services.

**Installed packages:**
- `google-api-python-client` – Google Docs API client
- `google-auth` – Authentication support
- `google-auth-oauthlib` – OAuth helpers
- `markdown` – Markdown parsing utilities

These packages are required only once per Colab runtime.


In [1]:
!pip install --quiet google-api-python-client google-auth google-auth-oauthlib markdown


### Google Authentication

This cell authenticates your Google account inside Colab so the notebook
can create and modify Google Docs on your behalf.

You will be prompted to:
1. Choose a Google account
2. Grant permission to access Google Docs

This authentication is required **once per session**.


In [2]:
from google.colab import auth
auth.authenticate_user()


### Markdown → Google Docs Conversion Module

This cell defines the reusable `MarkdownToGoogleDoc` class.

#### Responsibilities:
- Parse Markdown content
- Detect:
  - Headings (`#`, `##`, `###`)
  - Bullet points
  - Checkboxes (`- [ ]`)
  - Action Items
  - Footers
- Convert parsed content into:
  - Google Docs headings
  - Bulleted lists
  - **Functional Google Docs checkboxes**
- Apply styling:
  - Headings
  - Mention highlighting (`@username`)
  - Italics for footer metadata

#### Design Highlights:
- Clean separation of concerns
- Defensive error handling
- Logging instead of print statements
- Stateless per document (safe for batch processing)

This module is written to `meeting_to_gdoc.py` so it can be imported
and reused across notebooks.


In [5]:
%%writefile meeting_to_gdoc.py
import re
import logging
from typing import Optional, List, Dict
from googleapiclient.discovery import build
from google.auth import default
from google.colab import auth

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")


class MarkdownToGoogleDoc:
    def __init__(self, title: str = "Parsed Meeting Notes"):
        self.title = title
        self.docs_service = None
        self.document_id: Optional[str] = None

        # Reset per document
        self.cursor: int = 1
        self.requests: List[Dict] = []

        self.action_item_heading = "Action Items"

    # ------------------ Auth & Doc Lifecycle ------------------ #
    def authenticate(self) -> None:
        """Authenticate with Google Docs API (Colab)."""
        try:
            auth.authenticate_user()
            creds, _ = default(scopes=["https://www.googleapis.com/auth/documents"])
            self.docs_service = build("docs", "v1", credentials=creds)
        except Exception as e:
            logging.error(f"Authentication failed: {e}")
            raise

    def create_document(self, title: Optional[str] = None) -> None:
        """Create a new Google Doc."""
        if not self.docs_service:
            raise RuntimeError("Docs service not initialized. Call authenticate() first.")

        try:
            title = title or self.title
            doc = self.docs_service.documents().create(body={"title": title}).execute()
            self.document_id = doc["documentId"]
            logging.info(f"Created document: https://docs.google.com/document/d/{self.document_id}")
        except Exception as e:
            logging.error(f"Failed to create document: {e}")
            raise

    def _reset_state(self) -> None:
        """Reset cursor and request buffer for a new document."""
        self.cursor = 1
        self.requests.clear()

    # ------------------ Markdown Parsing ------------------ #
    def parse_markdown(self, md_text: str) -> List[Dict]:
        """Convert markdown text into structured tokens."""
        result = []
        current_section = None

        for raw_line in md_text.splitlines():
            line = raw_line.rstrip()
            if not line:
                continue

            if line.startswith("Meeting recorded by:") or line.startswith("Duration:"):
                result.append({"type": "footer", "text": line})
                continue

            heading_match = re.match(r"^(#{1,3})\s+(.*)", line)
            if heading_match:
                level = len(heading_match.group(1))
                current_section = heading_match.group(2)
                result.append({"type": "heading", "level": level, "text": current_section})
                continue

            checkbox_match = re.match(r"^(\s*)- \[ \]\s+(.*)", line)
            if checkbox_match:
                indent = len(checkbox_match.group(1))
                nesting = 0 if current_section == self.action_item_heading else indent // 2
                result.append({"type": "checkbox", "text": checkbox_match.group(2), "nesting": nesting})
                continue

            bullet_match = re.match(r"^(\s*)-\s+(.*)", line)
            if bullet_match:
                indent = len(bullet_match.group(1))
                result.append({"type": "bullet", "text": bullet_match.group(2), "nesting": indent // 2})
                continue

            result.append({"type": "text", "text": line})

        return result

    # ------------------ Google Docs Helpers ------------------ #
    def _insert_text(self, text: str) -> tuple[int, int]:
        start = self.cursor
        self.requests.append({
            "insertText": {
                "location": {"index": start},
                "text": text + "\n"
            }
        })
        self.cursor += len(text) + 1
        return start, self.cursor

    def _apply_heading(self, start: int, end: int, level: int) -> None:
        self.requests.append({
            "updateParagraphStyle": {
                "range": {"startIndex": start, "endIndex": end},
                "paragraphStyle": {"namedStyleType": f"HEADING_{level}"},
                "fields": "namedStyleType"
            }
        })

    def _apply_bullets(self, start: int, end: int, checkbox: bool, nesting: int) -> None:
        payload = {
            "createParagraphBullets": {
                "range": {"startIndex": start, "endIndex": end},
                "bulletPreset": "BULLET_CHECKBOX" if checkbox else "BULLET_DISC_CIRCLE_SQUARE"
            }
        }
        if not checkbox and nesting > 0:
            payload["createParagraphBullets"]["nestingLevel"] = nesting
        self.requests.append(payload)

    def _style_mentions(self, start: int, end: int, text: str) -> None:
        if re.search(r"@\w+", text):
            self.requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": start, "endIndex": end},
                    "textStyle": {
                        "bold": True,
                        "foregroundColor": {
                            "color": {"rgbColor": {"red": 0.2, "green": 0.2, "blue": 0.8}}
                        }
                    },
                    "fields": "bold,foregroundColor"
                }
            })

    # ------------------ Core Processing ------------------ #
    def add_block(self, text: str, *, heading=None, checkbox=False, nesting=0) -> None:
        try:
            start, end = self._insert_text(text)

            if heading:
                self._apply_heading(start, end, heading)

            if checkbox or nesting > 0:
                self._apply_bullets(start, end, checkbox, nesting)

            if checkbox:
                self._style_mentions(start, end, text)

        except Exception as e:
            logging.error(f"Failed to add block '{text}': {e}")

    def process_parsed(self, parsed: List[Dict]) -> None:
        for item in parsed:
            if item["type"] == "heading":
                self.add_block(item["text"], heading=item["level"])
            elif item["type"] == "checkbox":
                self.add_block(item["text"], checkbox=True, nesting=item["nesting"])
            elif item["type"] == "bullet":
                self.add_block(item["text"], nesting=item["nesting"])
            elif item["type"] == "footer":
                start, end = self._insert_text(item["text"])
                self.requests.append({
                    "updateTextStyle": {
                        "range": {"startIndex": start, "endIndex": end},
                        "textStyle": {
                            "italic": True,
                            "foregroundColor": {
                                "color": {"rgbColor": {"red": 0.4, "green": 0.4, "blue": 0.4}}
                            }
                        },
                        "fields": "italic,foregroundColor"
                    }
                })
            else:
                self.add_block(item["text"])

    def generate_document(self, md_text: str) -> None:
        if not self.docs_service or not self.document_id:
            raise RuntimeError("Document not initialized correctly")

        try:
            self._reset_state()
            parsed = self.parse_markdown(md_text)
            self.process_parsed(parsed)

            self.docs_service.documents().batchUpdate(
                documentId=self.document_id,
                body={"requests": self.requests}
            ).execute()

            logging.info("Document generation complete")

        except Exception as e:
            logging.error(f"Document generation failed: {e}")
            raise


Overwriting meeting_to_gdoc.py


### Upload and Convert Markdown Files

This cell:
1. Prompts the user to upload one or more `.md` files
2. Reads each Markdown file
3. Creates a new Google Doc for each file
4. Converts Markdown content into a formatted Google Doc
5. Prints the direct Google Docs link after creation

#### Features:
- Supports multiple Markdown files in one run
- Graceful handling of upload or processing errors
- Clear logging for each processed file

Each Markdown file becomes **one Google Doc**.


In [7]:
from google.colab import files
from meeting_to_gdoc import MarkdownToGoogleDoc
import logging

def upload_and_convert_markdown() -> None:
    uploaded_files = files.upload()

    if not uploaded_files:
        logging.warning("No files uploaded.")
        return

    for filename in uploaded_files:
        logging.info(f"Processing file: {filename}")

        try:
            with open(filename, "r") as f:
                markdown_text = f.read()

            generator = MarkdownToGoogleDoc(title="Meeting Notes")
            generator.authenticate()
            generator.create_document()
            generator.generate_document(markdown_text)

            print(
                f"Google Doc created: "
                f"https://docs.google.com/document/d/{generator.document_id}"
            )

        except Exception as e:
            logging.error(f"Failed to process {filename}: {e}")

upload_and_convert_markdown()


Saving meeting_notes.md to meeting_notes (3).md


✅ Google Doc created: https://docs.google.com/document/d/1erf_b3q5rpR2CP6DMKDMt1wEuSsbEWL8S-8fUWFFWTI
